# Week13 Hands-On: Char-level LSTM Text Generation

**TensorFlow / Keras — Full version (15 epochs)**

This hands-on notebook trains a small char-level LSTM on a short sample corpus. Replace `text_corpus` with a larger file (e.g., Shakespeare) for better results.

In [ ]:

# Char-level LSTM text generation — Hands-On
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np, os, random
import io

# Small sample corpus (replace with larger text file for real training)
text_corpus = (
    "To be, or not to be, that is the question:\n"
    "Whether 'tis nobler in the mind to suffer\n"
    "The slings and arrows of outrageous fortune,\n"
    "Or to take arms against a sea of troubles\n"
    "And by opposing end them."
)

print("Corpus length:", len(text_corpus))

# Character mapping
chars = sorted(list(set(text_corpus)))
char2idx = {c:i for i,c in enumerate(chars)}
idx2char = {i:c for c,i in char2idx.items()}

# Prepare sequences
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text_corpus) - maxlen, step):
    sentences.append(text_corpus[i: i+maxlen])
    next_chars.append(text_corpus[i+maxlen])
print("Number of sequences:", len(sentences))

# Vectorization
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.float32)
y = np.zeros((len(sentences), len(chars)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    for t, ch in enumerate(sentence):
        X[i, t, char2idx[ch]] = 1.0
    y[i, char2idx[next_chars[i]]] = 1.0

# Build model
model = models.Sequential([
    layers.LSTM(128, input_shape=(maxlen, len(chars))),
    layers.Dense(len(chars), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train (15 epochs as requested)
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, monitor='loss', restore_best_weights=True)]
history = model.fit(X, y, batch_size=16, epochs=15, callbacks=callbacks, verbose=2)

# Sampling utilities
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generate text
start_index = random.randint(0, max(1, len(text_corpus) - maxlen - 1))
generated = ""
seed = text_corpus[start_index: start_index + maxlen]
generated += seed
print("----- Seed: '" + seed + "'")
for temperature in [0.5, 1.0, 1.2]:
    print("------ temperature:", temperature)
    sys_stdout = ""
    gen = seed
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, ch in enumerate(gen[-maxlen:]):
            x_pred[0, t, char2idx[ch]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx2char[next_index]
        gen += next_char
    print(gen)
    
# Save model
outpath = os.path.join("/mnt/data", "week13_textgen_hands_on_model.h5")
model.save(outpath)
print("Saved model to", outpath)
